In [1]:
import warnings
from importlib import reload

warnings.filterwarnings('ignore')

In [50]:
from clients.gpt_database_logger import GPTDatabaseLogger
import re
import json

GPT_DB_LOGGER_PATH = 'sqlite:///prompt_versions.db'

db_logger = GPTDatabaseLogger(GPT_DB_LOGGER_PATH)

AGENT_API_URL = 'http://localhost:8013'
AGENT_API_URL = 'https://neon-dev.us'

In [11]:
import sys

sys.path.append('/Users/valuamba/projs/components_agent_sales/app/utils')

from html_messages_parser import get_element_messages, get_messages_from_html_file  


In [1]:
from typing import List

def get_conversation_html(file_name: str):
    with open(f'./deals_html/discount_v4/{file_name}.html', 'r') as f:
        return f.read()

def parse_file_name(file_name: str):
    deal_id, message_id = file_name.split('_', 1)
    return deal_id, message_id

def get_conversation_messages(file_name: str) -> List[str]:
    return get_messages_from_html_file(f'./deals_html/discount_v3/{file_name}.html')

def messages_to_pretty_format(messages: List[str]):
    for idx, msg in enumerate(messages):
        print(f'Message: {len(messages) - idx}')
        print(f'```\n' + msg + '\n```\n\n')

def select_json_block(text: str):
    match = re.search(r"```json\n([\s\S]*?)\n```", text)
    if match:
        json_data = match.group(1)
    else:
        raise ValueError("No valid JSON data found in the string.")

    return json.loads(json_data)


def prepare_conversation_to_prompt(messages_from_latest):
    messages_str = ''
    for idx, msg in enumerate(messages_from_latest):
        messages_str += f'Message: {len(messages_from_latest) - idx}\n'
        if 'Offer-Nr.' in msg:
            messages_str += '<Offer from CRM>'
        messages_str += f'```\n' + msg + '\n```\n\n'
    return messages_str

def filter_messages(messages_from_latest, skip: int, lenght: int):
    return messages_from_latest[::-1][skip:skip+lenght][::-1]
    

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(get_conversation_html('451540_41673.0'), 'html.parser')


soup.get_text()

'Dear Mr.Pushkar,With reference to the receipt of your Quotation Ref 451540, we request you to send us your final discounted Quotation addressed to Fire Shield Metal Works, Zone 57, Building 16, Industrial Area, Doha, QatarWe request you to confirm us that you will provide us the COC 2.1 certificate for the proposed productBest regardsThomas Philip (Manager)FIRE SHIELD METAL WORKSPost Box : 8318,Doha, Qatar.Tel : +974 44416569Mobile : +974 55549206Website :\xa0www.safari-industries.comOn Mon, Nov 6, 2023 at 9:38\u202fAM <pushkar@famaga.com> wrote:\n\n\n\n\nOffer-Nr.:\n\n                    451540                \n\nCustomer request #:\n\n\n\nCustomer #:\n\n                    48049                \n\n\nDate:\n\n                    Monday, 06 November 2023                \n\nInquiry #:\n\n\n\nContact person:\n\n                                            Pushkar Agarwal                                    \n\n\nOffer valid till\n\n                    06.12.2023                \n\nInquiry

In [7]:
file_name = '410158_CAFidWpS2O2JN6fPtwQ0SiqdsSG+y9qDr0xEWeZj_c==rEhG62g@mail.gmail.com'


In [8]:
conversation_html = get_conversation_html(file_name)

In [9]:
deal_id, message_id = parse_file_name(file_name)

In [25]:
messages = get_conversation_messages(file_name)

In [26]:
messages_from_start = messages[::-1]

In [ ]:
messages_to_pretty_format(messages_from_start[8:8+3][::-1])

In [114]:
def discount_processing(conversation: str) -> str:
    response = db_logger.create_completion(temperature=0.1, tags='discount_state', model='gpt-4', messages=[
        { "role": "user", "content": f"""
Analyze the messages and try to figure out discount state at conversation.

By 'discount,' I also mean a request to lower the price, etc. The following or other terms may be used for this purpose: - discount, payment terms, lower price, 
your price is too high, provide us a cheaper price, better price, expensive.

The possible discount states: NotHandled, Rejected, RepeatedRequest, NotAsked.

Conversation:
{conversation}

Put the response into ```json``` format like this:
```json
{{
    "state": <discount state>,
    "justification": <justification why client ask for discount> // fill this field only if it specified at customer messages
    "messages_ids": array[] // messages ids where customer or sales manager discuss or mention discount
}}
```
"""}])
    discount_state = select_json_block(response)
    return discount_state
    

In [115]:
prepared_messages = filter_messages(messages, 8, 3)
prepared_conversation = prepare_conversation_to_prompt(prepared_messages)


In [116]:
response = discount_processing(prepared_conversation)

```json
{
    "state": "NotHandled",
    "justification": "The customer has already made the payment and is asking for a discount.",
    "messages_ids": [3]
}
```

--------------------

Note saved without feedback. ID: 5afaf4a8-4b0d-4a2d-bc08-345562027955
Input tokens: 861 Output tokens: 42 Total price: 0.03$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [ ]:
print(prepared_conversation)

In [118]:
discount_messages = ''
for id in sorted(response['messages_ids'], reverse=True):
    discount_messages += f'Message: {id}\n'
    discount_messages += f'```\n' + prepared_messages[-id] + '\n```\n\n'

In [63]:
print(prepared_messages[-response['messages_ids'][0]])

good afternoon,
                                                  please could you give
                                                  me a discount for
                                                  already made the
                                                  payment and can you
                                                  tell me how long it
                                                  takes for me to
                                                  receive it in
                                                  Mozambique?<kg1@famaga.de> escreveu em qui.,
                                                  15/06/2023 às 14:44 :


In [99]:
import requests
from collections import defaultdict
from datetime import datetime

class FamagaClient:
    def __init__(self, api_key, session_id):
        self.base_url = "https://api.famaga.org"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Cookie": f"PHPSESSID={session_id}"
        }

    def list_offers_by_deal_id(self, deal_id):
        """List offers by deal ID"""
        response = requests.get(f"{self.base_url}/requisition?request={deal_id}", headers=self.headers)
        return response.json()

    def parts_selected_at_offer(self, offer_id):
        """Fetch parts selected at a specific offer"""
        response = requests.get(f"{self.base_url}/requisition/{offer_id}/products", headers=self.headers)
        return response.json()

    def list_offers_by_client_id(self, client_id):
        """Retrieve offers by client ID"""
        response = requests.get(f"{self.base_url}/requisition?client={client_id}", headers=self.headers)
        return response.json()

    def list_current_offer_details(self, offer_id):
        """List details of the current offer in a specific format"""
        parts = self.parts_selected_at_offer(offer_id)
        formatted_parts = []
        for part in parts:
            formatted_part = f"({part['articul']} {part['brand_title']}) margin: {part['margin_ru']}%, sell: {part['price_end']}$ qty. {part['count']}"
            formatted_parts.append(formatted_part)
        return "\n".join(formatted_parts)

    def find_parts_client_bought_before(self, client_id, current_offer_parts):
        """Return the parts that a client has bought before"""
        all_offers = self.list_offers_by_client_id(client_id)
        past_parts = []
        for offer in all_offers.get('content', []):
            parts = self.parts_selected_at_offer(offer['request']['id'])
            past_parts.extend(parts)
        
        bought_before = []
        current_parts_articul = [part['articul'] for part in current_offer_parts]
        for part in past_parts:
            if part['articul'] in current_parts_articul:
                bought_before.append(part)
        
        return bought_before

    def get_client_purchase_history_formatted(self, client_id, exluded_deal: int=None):
        """Generate formatted purchase history for a client"""
        history_data = self.list_offers_by_client_id(client_id)
        purchases = defaultdict(list)
        
        for item in history_data.get('content', []):
            date_key = datetime.strptime(item['created'], '%Y-%m-%dT%H:%M:%S%z').date()
            request_id = int(item['request']['id'])
            if exluded_deal is None or request_id != exluded_deal:
                parts = self.parts_selected_at_offer(request_id)
    
                for part in parts:
                    line = f"{item['id']} ({part['articul']} {part['brand_title']}) margin: {part['margin_ru']}%, sell: {part['price_end']}$ qty. {part['count']}, request id {part['request_id']}"
                    purchases[date_key].append(line)
        
        # Formatting the result
        result = "[CLIENT PURCHASE HISTORY]\n**Client purchase history:**\n"
        for date in sorted(purchases):
            result += f"Date: {date}\n"
            result += "\n".join(purchases[date]) + "\n\n"
        result += "[/CLIENT PURCHASE HISTORY]"
        return result




In [100]:
api_key = "YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw"
session_id = "085qpt4eflu39a0dg7hjhr5mdu"
client = FamagaClient(api_key, session_id)

In [68]:
offer_info = client.list_offers_by_deal_id(deal_id)
offer_id = offer_info['content'][0]['request']['id']
client_id = offer_info['content'][0]['request']['firm']['id']

In [109]:
purchase_history = client.get_client_purchase_history_formatted(client_id, int(deal_id))

In [119]:
current_offer = client.list_current_offer_details(offer_id)

In [124]:
response = db_logger.create_completion(temperature=0, tags='query_builder', model='gpt-4', messages=[
        { "role": "user", "content": f"""
Select one of documents with instructions:

- Discount -> Default: default instruction how to do negotiations with customer about decision and pricing.
- Discount -> Purchased same product: guide how to deal with loyal customer that already bought product that requires at current deal.

Messages about discount at conversation:
```
{discount_messages}
```

{purchase_history}

Deal offer:
{current_offer}

Answer should contain single document name with no other text.
    
"""}])

Discount -> Default

--------------------

Note saved without feedback. ID: cd3dfa1c-5fce-44b5-9226-0674bd1adb12
Input tokens: 228 Output tokens: 3 Total price: 0.01$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [125]:
document_name = response

In [105]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pydantic import BaseModel, Field, field_validator


class GoogleSheetToSQLiteConverter:
    def __init__(self, spreadsheet: str, credentials_path: str):
        scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        self.spreadsheet = spreadsheet
        self.client = gspread.authorize(creds)
        
    def get_sheet_data(self, sheet: str):
        worksheet = self.client.open(self.spreadsheet).worksheet(sheet)
        records = worksheet.get_all_values()

        return records


ghconv = GoogleSheetToSQLiteConverter(
    spreadsheet='Famaga Knowledge Map', 
    credentials_path=r'langchain-400510-06936d0d30b5.json')




In [127]:
sheet_data = ghconv.get_sheet_data(document_name)

In [128]:
data = sheet_data

# 1. Construct the Field Names section
field_names_section = "#### 1. **Field Names**:\n"
for i in range(len(data[0]) - 1):  # Skip the "instruction" column
    field_name = data[0][i]
    field_description = data[1][i]
    field_names_section += f"   - **\"{field_name}\"**: {field_description}\n"

# 2. Construct the Possible Values section
possible_values_section = "#### 2. **Possible Values**:\n"
possible_values_section += "   - **TRUE**: The condition is met.\n"
possible_values_section += "   - **FALSE**: The condition is not met.\n"
possible_values_section += "   - **ANY**: The condition is not relevant or any value is acceptable.\n"

# Combine sections to form the final instruction
final_instruction = f"{field_names_section}\n{possible_values_section}"

print(final_instruction)


#### 1. **Field Names**:
   - **"Is desired discount specified?"**: Wether customer specified desired discount at percents or amount?
   - **"Is margin still above 10%?"**: If you apply desired discount wether margin still greater than 10%?

#### 2. **Possible Values**:
   - **TRUE**: The condition is met.
   - **FALSE**: The condition is not met.
   - **ANY**: The condition is not relevant or any value is acceptable.



In [129]:
response = db_logger.create_completion(temperature=0.1, tags='query_builder', model='gpt-4', messages=[
        { "role": "user", "content": f"""
Build query for this input data.

Messages about discount at conversation:
```
{discount_messages}
```

{purchase_history}
        
### Querying Instructions:

{final_instruction}

#### 3. **Building a Query**:
   - Use only field names that listed at instruction with possible values
   - Use the `query()` method in Pandas to filter the table based on the conditions.
   - Format each condition as a string using the field names and their desired values.

#### 5. **Example Queries**:

```json
{{
    "query": "`Is margin still above 10%?` == 'TRUE' and `Is the same margin on product as previous?` == 'TRUE'"
}}
```
"""}])

```json
{
    "query": "`Is desired discount specified?` == 'FALSE'"
}
```

--------------------

Note saved without feedback. ID: 6d1e1c46-8fa4-445e-b506-f0bb7f31f17c
Input tokens: 344 Output tokens: 21 Total price: 0.01$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [130]:
query = select_json_block(response)

In [131]:
import pandas as pd


headers = sheet_data.pop(0)
sheet_data.pop(0)
df = pd.DataFrame(sheet_data, columns=headers)

df

,Is desired discount specified?,Is margin still above 10%?,instruction
0,FALSE,ANY,Aks customer about desired discount
1,TRUE,TRUE,Apply customer desired discount
2,TRUE,FALSE,Ask about justificastions


In [138]:
result = df.query("`Is desired discount specified?` == 'ANY' and `Is margin still above 10%?` == 'FALSE'")
result

,Is desired discount specified?,Is margin still above 10%?,instruction


In [135]:
response = db_logger.create_completion(temperature=0.1, tags='execute_task', model='gpt-4', messages=[
        { "role": "user", "content": f"""
You are sales manager. Please do task by instruction:
{result['instruction'].iloc[0]}

The conversation with customer context:

Messages about discount at conversation:
```
{discount_messages}
```

{purchase_history}

Deal offer:
{current_offer}

Put result of task into ```json``` format, like this:
```json
{{
    "output": "the output of task"
}}
```
"""}])

```json
{
    "output": "Good afternoon, thank you for your purchase. I understand you are interested in a discount. Could you please specify the amount or percentage of discount you are looking for?"
}
```

--------------------

Note saved without feedback. ID: 95903d1d-79df-427f-a2f9-3e477dfd95ba
Input tokens: 218 Output tokens: 44 Total price: 0.01$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…